In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pydicom
import xarray as xr

In [ ]:
from pymedphys._data import download
from pymedphys._dicom import constants, coords, dose

import xdose

In [ ]:
dicom_dose_filepaths = {orient: download.get_file_within_data_zip(
    "dicomorient_doses_only.zip", f"RD.DICOMORIENT.Dose_{orient}.dcm"
) for orient in constants.IMAGE_ORIENTATION_MAP}

rtdoses_by_orient = {orient: pydicom.dcmread(dicom_dose_filepaths[orient]) for orient in constants.IMAGE_ORIENTATION_MAP}

In [ ]:
xdoses_by_orient = {orient: xdose.xdose_from_dataset(rtdoses_by_orient[orient], name=f"Dose {orient}", coord_system="S")  for orient in constants.IMAGE_ORIENTATION_MAP if orient == "HFS"}

xdoses_by_orient["HFS"]

In [ ]:
# Need method="nearest" if exact coordinate isn't suppled to sel():
try:
    xdoses_by_orient["HFS"].sel(x=0)
except KeyError:
    print("KeyError")

In [ ]:
xdoses_by_orient["HFS"].sel(x=0, method="nearest")

In [ ]:
# Value rounding

print(f"\nValues unrounded:\n{xdoses_by_orient['HFS'].max()}")
print(f"\nX coords unrounded:\n{xdoses_by_orient['HFS'].x.max()}")
print(f"\nValues rounded:\n{xdoses_by_orient['HFS'].round(decimals=2).max()}")
print(f"\nX coords aren't rounded (only values are rounded):\n{xdoses_by_orient['HFS'].round(decimals=2).x.max()}")
print(f"\nValues unrounded again (rounding on previous lines weren't 'in_place'):\n{xdoses_by_orient['HFS'].max()}")

In [ ]:
# Coordinate rounding

print(f"\nX coords unrounded:\n{xdoses_by_orient['HFS'].x.max()}")
print(f"\nX coords rounded:\n{xdoses_by_orient['HFS'].x.round(decimals=2).max()}")
print(f"\nX coords unrounded again (rounding on previous line wasn't 'in_place'):\n{xdoses_by_orient['HFS'].x.max()}")

In [ ]:
# Very easily plot dose slice, already labelled with colorbar and units
def plot_xdose_tcs_at_point(xdose_to_plot, point, coord_system="S"):
    LAT_2_LONG_RATIO = xdose_to_plot.x.size / xdose_to_plot.y.size   
    VERT_2_LONG_RATIO = xdose_to_plot.z.size / xdose_to_plot.y.size

    xdose_to_plot = xdose.round_xdose_coords(xdose_to_plot)

    fig, axes = plt.subplots(figsize=(12, 8), ncols=2, nrows=2, gridspec_kw={"width_ratios":(LAT_2_LONG_RATIO,1), "height_ratios":(VERT_2_LONG_RATIO,1)})
    axes[1, 1].axis('off')
    for ax in axes.ravel():
        ax.set_aspect("equal")

    try:
        xdose_to_plot.sel(y=point[1], method="nearest").plot(ax=axes[0, 0], cmap="jet", vmin=0, vmax=xdose_to_plot.max())
        axes[0, 0].axhline(y=point[2], color="silver")
        axes[0, 0].axvline(x=point[0], color="silver")

        xdose_to_plot.sel(x=point[0], method="nearest").T.plot(ax=axes[0, 1], cmap="jet", vmin=0, vmax=xdose_to_plot.max())
        axes[0, 1].axhline(y=point[2], color="silver")
        axes[0, 1].axvline(x=point[1], color="silver")

        xdose_to_plot.sel(z=point[2], method="nearest").plot(ax=axes[1, 0], cmap="jet", vmin=0, vmax=xdose_to_plot.max())
        axes[1, 0].axhline(y=point[1], color="silver")
        axes[1, 0].axvline(x=point[0], color="silver")
    except ValueError:
        pass

    fig.tight_layout()

In [ ]:
# Plot at point max
argmax = xdoses_by_orient["HFS"].argmax(...)

point_max = (
    xdoses_by_orient["HFS"].x[argmax['x']],
    xdoses_by_orient["HFS"].y[argmax['y']],
    xdoses_by_orient["HFS"].z[argmax['z']]
)

plot_xdose_tcs_at_point(xdose_to_plot=xdoses_by_orient["HFS"], point=point_max, coord_system="S")

In [ ]:
# Plot HFS cup at centre

plot_xdose_tcs_at_point(xdose_to_plot=xdoses_by_orient["HFS"], point=(0, -1157, 0))

In [ ]:
# Zoom!

x_start = -55
x_end = 55
y_start = -1225
y_end = -1100
z_start = 60
z_end = -60
xdose_hfs_zoomed = xdoses_by_orient['HFS'].sel(x=slice(x_start, x_end), y=slice(y_start, y_end), z=slice(z_start, z_end))
plot_xdose_tcs_at_point(xdose_to_plot=xdose_ffdl_zoomed, point=(0, -1157, 0))

In [ ]:
# Lovely inbuilt interpolation too!

x_new = np.linspace(xdose_hfs_zoomed.x[0], xdose_hfs_zoomed.x[-1], xdose_hfs_zoomed.sizes["x"]*4) # Make 4x finer
y_new = np.linspace(xdose_hfs_zoomed.y[0], xdose_hfs_zoomed.y[-1], xdose_hfs_zoomed.sizes["y"]*4) # Make 4x finer
z_new = np.linspace(xdose_hfs_zoomed.z[0], xdose_hfs_zoomed.z[-1], xdose_hfs_zoomed.sizes["z"]*4) # Make 4x finer

xdose_hfs_interp = xdose_hfs_zoomed.interp(x=x_new, y=y_new, z=z_new)

In [ ]:
plot_xdose_tcs_at_point(xdose_to_plot=xdose_hfs_interp, point=(0, -1157, 0))